In [18]:
#CUT LiDAR INTO IMAGE SEGMENTS
#With a t,x,y,z Dataframe loaded representing the Chiroptera LiDAR,
#cuts the LiDAR into individual segments for each georeferenced raster image.

import os
from osgeo import gdal

unshifted_path = 'shifted_imagery/Chirop 1D/unshifted'

for i,filename in enumerate(list(os.listdir(unshifted_path))):
    if(filename.endswith('.tif')):
        full_unshifted_path = os.path.join(unshifted_path, filename)
        ds_original = gdal.Open(full_unshifted_path, gdal.GA_Update)
        # get the geotransform as a tuple of 6
        gt = ds_original.GetGeoTransform()
        x_tl, x_res, dx_dy, y_tl, dy_dx, y_res = gt
        (x_br, y_br) = x_tl + ds_original.RasterXSize * x_res, y_tl + ds_original.RasterYSize * y_res
        #print(f"({x_tl} {y_tl}) ({x_br} {y_br})")
        image_lidar = df[(df['x'] > x_tl) & (df['x'] < x_br) & (df['y'] < y_tl) & (df['y'] > y_br)]

        with open(f'shifted_imagery/Chirop 1D/unshifted lidar/{filename}.txt', 'w') as ofile:
            fmt = '%.0f %8.8f %8.2f %8.2f %8.12f'
            np.savetxt(ofile, image_lidar.values, fmt=fmt)

In [3]:
#SHIFT CHIROPTERA CSV BY EASTING AND NORTHING
#Given X and Y shift values (in meters), shifts a dataframe by those values.
#This function is unnecessary and requires modifying a lot of data.
#We can shift the ATL line instead.

chiroptera_path = 'data/Chiroptera_1B/0m_4m.txt'
shiftX, shiftY = 0,0

import pandas as pd
import os
import numpy as np

#'data/Chirop_FL2_SA/5-6mc.txt'
def shiftChiroptera(filepath, shiftX, shiftY):
    columns = ["c", "GPS_time", "Shape@X", "Shape@Y", "chiroptera_z"]
    df = pd.read_fwf(filepath, names=columns)
    df["Shape@X"] = df["Shape@X"] + shiftX
    df["Shape@Y"] = df["Shape@Y"] + shiftY
    fn, _ = filepath.split(".")

    with open(f"{fn}_{shiftX:.3f}_{shiftY:.3f}.txt", 'w') as ofile:
        fmt = '%.0f %8.8f %8.2f %8.2f %8.12f'
        np.savetxt(ofile, df.values, fmt=fmt)

shiftChiroptera(chiroptera_path, shiftX, shiftY)

In [7]:
#APPLY ROTATION AND SHIFT TO ATL CSV
import pandas as pd


from pyproj import CRS
from pyproj import Transformer
import pyproj


def convert_df_coords(df, epsg):
    crs = CRS.from_epsg(epsg)
    proj = Transformer.from_crs(crs.geodetic_crs, crs, always_xy=True)
    
    def transform_x(long, lat):
        return proj.transform(long, lat)[0]

    def transform_y(long, lat):
        return proj.transform(long, lat)[1]

    df.insert(loc=0, column='x', value=df.apply(lambda row: transform_x(row['lons'], row['lats']), axis=1))
    df.insert(loc=1, column='y', value=df.apply(lambda row: transform_y(row['lons'], row['lats']), axis=1))
    df = df.drop(['lons', 'lats'], axis=1)
    return df

def rotate_from_center(x, y, ox, oy, degrees):
    xt,yt = x - ox, y - oy
    c = np.cos(np.radians(degrees))
    s = np.sin(np.radians(degrees))
    xr = xt * c - yt * s
    yr = xt * s + yt * c
    x, y = xr + ox, yr + oy
    return x, y

# ox, oy = 558938, 9460633